# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [44]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [38]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,alofi,-19.0595,-169.9187,26.94,65,40,6.17,NU,1699415750
1,1,sitka,57.0531,-135.3300,7.94,87,100,5.66,US,1699415750
2,2,avarua,-21.2078,-159.7750,24.03,60,20,7.72,CK,1699415750
3,3,hawaiian paradise park,19.5933,-154.9731,26.73,82,0,2.57,US,1699415750
4,4,jamestown,42.0970,-79.2353,5.77,80,100,5.66,US,1699415750


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng",
                                       "Lat",
                                       geo=True,
                                       tiles="OSM",
                                       size="Humidity",
                                       scale=0.25,
                                       color="City",
                                       frame_width=700,
                                       frame_height=500)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[city_data_df['Max Temp'] < 27]
ideal_cities = ideal_cities.loc[ideal_cities['Max Temp'] > 21]
ideal_cities = ideal_cities.loc[ideal_cities['Wind Speed'] < 4.5]
ideal_cities = ideal_cities.loc[ideal_cities['Cloudiness'] == 0]

# Drop any rows with null values
cleaned_ideal_cities = ideal_cities.dropna(how='any')

# Display sample data
cleaned_ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,hawaiian paradise park,19.5933,-154.9731,26.73,82,0,2.57,US,1699415750
57,57,medina,24.4686,39.6142,23.24,33,0,2.57,SA,1699415760
68,68,tura,25.5198,90.2201,25.02,56,0,0.66,IN,1699415763
139,139,el bauga,18.2620,33.9081,24.15,68,0,3.58,SD,1699415779
230,230,mandali,33.7499,45.5541,21.27,45,0,2.99,IQ,1699415799


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_ideal_cities.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,hawaiian paradise park,US,19.5933,-154.9731,82,
57,medina,SA,24.4686,39.6142,33,
68,tura,IN,25.5198,90.2201,56,
139,el bauga,SD,18.2620,33.9081,68,
230,mandali,IQ,33.7499,45.5541,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
hotel_df=hotel_df.reset_index(drop=True)

In [51]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'

params = {
    'categories':categories,
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=hotel_df.iloc[index,2]
    lng=hotel_df.iloc[index,3]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
medina - nearest hotel: The Oberoi Madina
tura - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
mandali - nearest hotel: No hotel found
khark - nearest hotel: No hotel found
aswan - nearest hotel: Yaseen hotel
manjhanpur - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
rockport - nearest hotel: Holiday Inn Express & Suites
badarganj - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
pitiquito - nearest hotel: Hotel Oasis


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,82,No hotel found
1,medina,SA,24.4686,39.6142,33,The Oberoi Madina
2,tura,IN,25.5198,90.2201,56,No hotel found
3,el bauga,SD,18.2620,33.9081,68,No hotel found
4,mandali,IQ,33.7499,45.5541,45,No hotel found
5,khark,IR,29.2614,50.3306,61,No hotel found
6,aswan,EG,24.0934,32.9070,41,Yaseen hotel
7,manjhanpur,IN,25.5333,81.3833,29,No hotel found
8,hilo,US,19.7297,-155.0900,75,Dolphin Bay Hotel
9,rockport,US,28.0206,-97.0544,87,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo=True,
                                    tiles="OSM",
                                    size="Humidity",
                                    scale=0.5,
                                    color="City",
                                    hover_cols=["Hotel Name","Country"],
                                    frame_width=750,
                                    frame_height=250)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)